In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import integrate as sci
import time
import pip
from getdist import plots, MCSamples
%matplotlib inline

In [3]:
# Este programa implementa Hamiltonian Markov Chain Monte Carlo. El núcleo utilizado se basa en Metropolis Hastings.

def likelihood(mod, dat, sigma): # retorna escalar, log(L)
    sig = np.diagonal(sigma)
    L = -0.5*chi2(mod, dat, sigma)[0]  + np.sum(-0.5*np.log(2*np.pi*sig**2))
    #pp = np.argwhere((a1==-np.inf))
    #a1[pp] = 0
    return L


def chi2(mod, dat, sigma):
    sig = np.diagonal(sigma)
    AA = np.sum(((dat - mod)/sig)**2)
    BB = np.sum((dat - mod)/sig**2)	
    CC = np.sum(1/sig**2)
    chi = AA - (BB**2)/CC
    return chi, BB/CC


def prior(theta): # log(pi)
    ct = 1
    r = np.diag(np.ones(len(theta))*ct)
    p = -0.5*np.log(np.linalg.det(2*np.pi*r)) - 0.5*theta.dot((np.linalg.inv(r)).dot(theta)) 
    return p


def acepta(ec, ep, EC, EP, x, X):
    alpha = min(- EP - EC + ep + ec, 0) # log(alpha)
    u = np.log(np.random.uniform())
    if u<alpha:
        return X, EP
    else:
        return x, ep


def roun(x, d):
    x = x*10**d
    x = np.int(np.rint(x))
    x = x*10**-d
    return x


def EHubble(theta, z): # parametro de hubble
    om0 = theta[0]
    ol = theta[1]
    w = theta[2]
    arg = om0*(1 + z)**3 + (1 - om0 - ol)*(1 + z)**2 + ol*(1 + z)**(3*(1 + w))
    EE = np.sqrt(arg)
    return EE, arg


def modelo(theta,z): # modulo de la distancia teorico
    om0 = theta[0]
    ol = theta[1]
    omega_k = 1 - om0 - ol
    E = EHubble(theta, z)[0]
    I = sci.cumtrapz(1/E, z, initial=0)+z[0]*((1/E)[0] + 1)/2
    o_k_s = np.sqrt(abs(omega_k))
    if omega_k==0:
        dl = (1 + z)*I
    elif omega_k<0:
        dl = (1 + z)*np.sin(o_k_s*I)/o_k_s
    elif omega_k>0:	
        dl = (1 + z)*np.sinh(o_k_s*I)/o_k_s	
    dist = 5*np.log10(dl)
    if (-np.inf==dist).any(): 
        print(theta)
    return dist


def tasa(tant, tpos):
    l = len(tant)
    if np.sum(tant==tpos)==l:
        c = 0
    else:
        c = 1
    return c


def revisa(theta, z):
    arg = EHubble(theta, z)[1]
    bol = np.sum(arg<0)
    if bol>0:
        a = 0 # raiz imaginaria 
    else:
        a = 1 # raiz real
    return a


def region(f, x, y, z, alpha, n):
    m = np.int(np.around(alpha*n,0))
    argsort = np.argsort(f)
    xs = x[argsort]
    ys = y[argsort]
    zs = z[argsort]
    xsa = xs[:m]
    ysa = ys[:m]
    zsa = zs[:m]
    return xsa, ysa, zsa


def argmin1(t1, t2, t3, z, dat, sig): # calcula el min om, ol, w en base a chi2	
    minchic = np.inf
    a = 0
    for i,j,k in zip(t1,t2,t3):
        mo = modelo(np.array([i,j,k]),z)
        chic = chi2(mo, dat, sig)[0]
        if chic<minchic:
            amin = a
            minchic = chic
        a += 1
    return t1[amin], t2[amin], t3[amin]


def argmin2(t1, t2, t3, V): # busca en base a vector chi2 y devulve minimos de los parametros	
    amin = np.argmin(V)
    return t1[amin], t2[amin], t3[amin]


def dx2(s, n_p):
    """
    s: sigmas
    n_p: numero de parametros
    """
    delta_x_2 = gammainccinv(n_p/2, erfc(s/np.sqrt(2)))
    return delta_x_2


def samples_s(sa1, sa2, x2, d_x2): # determina samples dentro de sigmas
    ind_sam = np.where(x2<=min(x2)+d_x2)
    return sa1[ind_sam], sa2[ind_sam]


def cuenta(x2, W):
    """
    recibe muestras de una distribucion y X2 para determinar los puntos y X2 para las regiones de credibilidad de 0.68, 0.95 y 0.99 
    """
    ms = W.shape[1]
    args = np.argsort(x2)
    W = W[:,args]  
    ms68 = int(np.rint(ms*0.68))
    ms95 = int(np.rint(ms*0.95))
    ms99 = int(np.rint(ms*0.99))
    R = np.array([])
    for wi in W:
        R = np.concatenate((R, np.array((wi[:ms68], wi[:ms95], wi[:ms99]))))
    return R, x2[ms68], x2[ms95], x2[ms99]


def potencial(dat, sigma, theta, z):
    mod = modelo(theta, z)
    u = - likelihood(mod, dat, sigma) - prior(theta) 
    return u


def cinetica(p, m):
    k = np.sum(p**2/2/m)
    return k


def hamiltoniano(p, dat, sigma, theta, z, m=1):
    h = cinetica(p, m) + potencial(dat, sigma, theta, z)
    return h


def gradiente(dw, theta, z, dat, sigma):
    tf = theta + 0.5*dw
    tb = theta - 0.5*dw
    grad = (potencial(dat, sigma, tf, z) - potencial(dat, sigma, tb, z))/dw
    return grad


def leapfrog(l, e, dw, theta, m, z, dat, sigma):
    qe = theta
    while True:	
        pi = np.random.multivariate_normal(mean=np.zeros(3), cov=np.diag(m))
        pe = pi
        X = []
        P = []
        X.append(theta)
        P.append(pe)
        for i in range(l):
            rev = revisa(qe, z)*revisa(qe+0.5*dw, z)*revisa(qe-0.5*dw, z)
            if rev==0:
                break
            pe = pe - 0.5*e*gradiente(dw, qe, z, dat, sigma) # actualiza momento en e/2	
            qe = qe + e*pe/m
            rev = revisa(qe, z)*revisa(qe+0.5*dw, z)*revisa(qe-0.5*dw, z)
            if rev==0:
                break
            pe = pe - 0.5*e*gradiente(dw, qe, z, dat, sigma)
            P.append(pe)
            X.append(qe)
            if i+1==l: i+=1
        if i==l: break
    P = np.array(P)
    X = np.array(X)
    return X, P

In [4]:
# install('getdist')

# saved things directory
direc = '/home/mauricio/Documents/Uni/Intro_2/' + 'gal.txt'

# Carga de datos
redshift = np.genfromtxt('gal.txt', usecols=(1))
mu_obs = np.genfromtxt('gal.txt', usecols=(2)) # m - M
cov = np.genfromtxt('gal.txt', usecols=(3))

p = np.argsort(redshift)
redshift = redshift[p]
mu_obs = mu_obs[p]
cov = cov[p]
cov = np.diag(cov)

In [8]:
# configuracion cadena

# params
labs = [r'$\Omega_{m}$', r'$\Omega_{\Lambda}$', r'w']
labs1 = [r'\Omega_{m}', r'\Omega_{\Lambda}', r'w']

M = 1 # numero de cadenas
Chains = []
Xi2 = []
Post = []
COV = []
params = 3 # numero de parametros
r = 0.75e-2 #0.75e-2
m = np.array([0.5, 0.5, 1])*r  # np.ones(3)*r # (np.array([0.4, 1.125, 4])**2)*1e-2  
pg = 1e-4 # paso gradiente # 0.5
ps = pg*10 # paso solver
'numero de parametros', params
# q inicial, revisa que sea valido
while True:
    q = np.random.uniform(low=[0,0,-5], high=[1, 2, 0], size=3)
    if revisa(q,redshift)==1: 
        break

print('params iniciales', q)
L = 5
print('pasos solver:', L)
N = 100
print('numero de muestras:', N)

params iniciales [ 0.86480929  1.37497063 -0.00694925]
pasos solver: 5
numero de muestras: 100


In [11]:
# Matrices de datos de la cadena
chain = [] 
post = [] 
chi_2 = []
Ratio = []
H = []
acept = 0
mod1 = modelo(q, redshift)
pos1 = potencial(mu_obs, cov, q, redshift)
Chi1 = chi2(mod1, mu_obs, cov)[0]
chain.append(q)
post.append(pos1)
chi_2.append(Chi1)
Ratio.append(100)

Ti = time.time()
for i in range(N):
    print(i)
    q = chain[i]
    # revision si proposal no indefine la busqueda
    while True:	
        Q, P = leapfrog(L, ps, pg, q, m, redshift, mu_obs, cov)	
        Q1 = Q[-1]
        P1 = P[-1]
        if revisa(Q1, redshift)==1: # que la raiz no sea imaginaria
            break
    H.append(Q)
    t = cinetica(P1[0], m)
    u = potencial(mu_obs, cov, q, redshift)
    T = cinetica(P1, m)
    U = potencial(mu_obs, cov, Q1, redshift)
    A = acepta(t, u, T, U, q, Q1)
    chain.append(A[0])
    post.append(A[1])
    mod1 = modelo(A[0], redshift)
    Chi1 = chi2(mod1, mu_obs, cov)[0]
    chi_2.append(Chi1)	
    # ratio de aceptacion
    acept += tasa(chain[i], chain[i + 1]) 
    Ratio.append(acept/(i+1)*100)

Tf = time.time()
print('Tiempo cadena', np.around(Tf - Ti, 0), 's')

ratio = acept/N*100
print('ratio %', np.rint(ratio))

post = np.array(post)
chain = np.array(chain)
chi_2 = np.array(chi_2)
Ratio = np.array(Ratio)
H = np.array(H)

t1 = chain[:,0]
t2 = chain[:,1]
t3 = chain[:,2]

# busca argumento del minimo de chi2
t1m, t2m, t3m = np.around(argmin2(t1, t2, t3, chi_2),3)
print('Xi2 minimo', t1m, t2m, t3m)

COV.append(covarianza)
Chains.append(chain)
Post.append(post)
Xi2.append(chi_2)

Chains = np.array(Chains)
Post = np.array(Post)
Xi2 =  np.array(Xi2)
COV = np.array(COV)


0


/home/mauricio/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:48: RuntimeWarning: invalid value encountered in sqrt


KeyboardInterrupt: 

## No esta corriendo, probar la derivada con programacion dinamica y grafos computacionales